In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from statistics import NormalDist
from scipy.stats import norm
import scipy.special as sp

# Downloading the data:

In [2]:
X = np.load(r"C:\Users\mo13\OneDrive\Documents\ML course\project\IMP-PCMLAI-capstone-initial_data\initial_data\function_2\initial_inputs.npy")
y = np.load(r"C:\Users\mo13\OneDrive\Documents\ML course\project\IMP-PCMLAI-capstone-initial_data\initial_data\function_2\initial_outputs.npy")

Here I'm appending the inputs resulted from the acquisition function below, so everytime I get an query, I add it here to the input "x"

In [3]:
X = np.append(X,[[0, 0.424242], [0, 0.363636], [0, 0], [0.272727, 0.999999]], axis=0)
X

array([[0.66579958, 0.12396913],
       [0.87779099, 0.7786275 ],
       [0.14269907, 0.34900513],
       [0.84527543, 0.71112027],
       [0.45464714, 0.29045518],
       [0.57771284, 0.77197318],
       [0.43816606, 0.68501826],
       [0.34174959, 0.02869772],
       [0.33864816, 0.21386725],
       [0.70263656, 0.9265642 ],
       [0.        , 0.424242  ],
       [0.        , 0.363636  ],
       [0.        , 0.        ],
       [0.272727  , 0.999999  ]])

Here I'm appending outputs I get from the feedback, so evertime I get the output from the feedback, I add it to the output "y"

In [4]:
y = np.append(y,[-0.011816, 0.04148, 0.115628206, -0.030545674], axis=0)
y

array([ 0.53899612,  0.42058624, -0.06562362,  0.29399291,  0.21496451,
        0.02310555,  0.24461934,  0.03874902, -0.01385762,  0.61120522,
       -0.011816  ,  0.04148   ,  0.11562821, -0.03054567])

# Fitting in Guassian Distribution

In [5]:
gpr = GaussianProcessRegressor()
gpr.fit(X, y);

# Creating a grid for grid search

In [6]:
x1 = np.linspace(0, 1, 100)
x2 = np.linspace(0, 1, 100)

In [7]:
X_grid = []
for i in range(len(x1)):
    for j in range(len(x2)):
        X_grid.append([x1[i], x2[j]])

Implementing the grid search:

In [8]:
X_grid = np.array(X_grid)
mean, std = gpr.predict(X_grid, return_std = True)


#####################################################################################################
#Below is my previous approach where I used the probability improvement method for the Acquisition Function (you can ignor it)
#ucb = mean + 1.96 * std
#acquisition_function = []
#eta=0.1
#mx = max([-0.011816, 0.04148])
#for k in range(len(X_grid)):
 #   acquisition_function.append(1 - NormalDist(mu = mean[k], sigma = std[k]).cdf(mx + eta))
  #  if  mean[k] > mx:
   #     mx = mean[k]
#acquisition_function = np.array(acquisition_function)

# Applying the "Analytic LogEI"

I have found this approach in "Unexpected Improvements to Expected Improvement for Bayesian Optimization" article (expalined in README file)

Now let's define log1mexp which is $$log( 1 - exp(x) )$$

In [9]:
def log1mexp(x):
    return np.log(1-np.exp(x))

$log( 1 - exp(x) )$ is used in the acquistion function later

Recall from README file: 

$$c1 = log(2π) / 2$$
$$c2 =  log(π/2) /2$$
$$erfcx(x) = exp(x^{2})erfc(x)$$
where erfc is the complementary error function.
The "Analytic LogEI" acquistion function is: $$ LogEIy^{∗}(x) = log_h((µ(x) − y^{∗})/σ(x)) + log(σ(x))$$
where $log_h(x)$ is calculated below (based on several conditions mentioned in README file and the article (section 4.1)

eta in the code below is ϵ which is chosen to be 0.1. So Let's apply that below:

In [10]:
z = (mean - max(y)) / std
c1 = np.log(2 * (22/7)) / 2
c2 = np.log((22/7) / 2) / 2
eta=0.1

if (z > -1).any:   # the first condition for log_h
    log_h = np.log(norm.pdf(z) + z * norm.cdf(z) + 1e-10)   # I added "1e-10" to avoid division by zero

elif (-1/np.sqrt(eta) < z).any() and (z <= -1).any():     # the second condition for log_h
    log_h = -z**2/(2-c1) + log1mexp(np.log(sp.erfcx(-z/np.sqrt(2)) * abs(z)) + c2)

elif (z <= -1 / np.sqrt(eta)).any():    # the third condition for log_h
    log_h = -z**2/(2-c1) + 2* np.log(abs(z))

acquisition_function = log_h + np.log(std)
acquisition_function

array([-34.53877746, -31.65940555, -31.02711761, ...,   0.83568366,
         0.89574325,   0.95666201])

# Obtaining the next Query:

In [11]:
idx_max = np.argmax(acquisition_function)
next_query = X_grid[idx_max]
print(next_query.round(6))

[1.       0.323232]
